In [ ]:
import feedparser
import httpx
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel
from pydantic import Field
from rich import print

In [2]:
load_dotenv()

True

In [5]:
url = "https://github.com/ava-labs/avalanchego/releases.atom"

resp = httpx.get(url)
resp.raise_for_status()


rss = feedparser.parse(resp.text)
for entry in rss["entries"]:
    for k, v in entry.items():
        print(f"{k}: {str(v)[:100]}...")
    break

id: tag:github.com,2008:Repository/246387644/v1.12.0-fuji...

guidislink: True...

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-fuji...

updated: 2024-11-19T02:31:48Z...

updated_parsed: time.struct_time(tm_year=2024, tm_mon=11, tm_mday=19, tm_hour=2, tm_min=31, tm_sec=48, tm_wday=1, 
tm...

...

authors: [{'name': 'StephenButtolph'}]...

author_detail: {'name': 'StephenButtolph'}...

author: StephenButtolph...

media_thumbnail: [{'height': '30', 'width': '30', 'url': 
'https://avatars.githubusercontent.com/u/22109487?s=60&v=4'}...

href: ...

In [ ]:
max_length = 5000
model = "gpt-4o-mini"
temperature = 0
max_tokens = 1000


class Hardfork(BaseModel):
    hardfork: bool = Field(..., description="Whether this is a hardfork")
    confidence: float = Field(..., description="Confidence in the prediction, between 0 and 1")


def predict_hardfork(text: str) -> Hardfork:
    client = OpenAI()
    response = client.beta.chat.completions.parse(
        messages=[
            {"role": "user", "content": text},
        ],
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        response_format=Hardfork,
    )

    if not response.choices:
        return Hardfork(hardfork=False, confidence=0)

    return response.choices[0].message.parsed


for entry in rss["entries"]:
    text = str(entry)
    hf = predict_hardfork(text)

    link = entry["link"]
    print(f"link: {link}, hardfork: {hf}")

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-fuji, hardfork: hardfork=True confidence=0.9

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13, hardfork: hardfork=False confidence=0.9

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.2, hardfork: hardfork=False confidence=0.8

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.1, hardfork: hardfork=False confidence=0.8

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.0, hardfork: hardfork=False confidence=0.9

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.9, hardfork: hardfork=False 
confidence=0.8

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.8, hardfork: hardfork=False 
confidence=0.8

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.7, hardfork: hardfork=False 
confidence=0.8

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.6, hardfork: hardfork=False 
confidence=0.9

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.12, hardfork: hardfork=False confidence=0.9